In [302]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [303]:
def datetimer(string):                    # чтобы даты в нормальный вид приводить
    month_dict = {
    'янв': '01',
    'фев': '02',
    'мар': '03',
    'апр': '04',
    'мая': '05',
    'июн': '06',
    'июл': '07',
    'авг': '08',
    'сен': '09',
    'окт': '10',
    'ноя': '11',
    'дек': '12'
    }
    date = string.split()
    day = int(date[0])
    year = int(date[2])
    month = int(month_dict[date[1]])
    date = datetime(year, month, day)
    return date

# табличка с погодой:

Мы взяли данные о погоде в Москве (метеостанция 65518, в районе ВДНХ) в ноябре 2014 - августе 2015 с сайта https://rp5.ru/Архив_погоды_в_По . Из полученного датасета мы используем следующие признаки:

- Дата
- Температура воздуха в градусах Цельсия (T)
- Атмосферное давление в миллиметрах ртутного столба (Po)
- Относительная влажность в % (U)
- Направление ветра - категориальный признак (DD)
- Скорость ветра в метрах в секунду (Ff)
- Общая облачность (N)
- Горизонтальная дальность видимости в км (VV)
- Температура точки росы в градусах Цельсия (Td)

In [304]:
df = pd.read_excel('xls_weather.xls')[['Местное время в Москве (ВДНХ)', 'T', 'Po', 'U', 'DD', 'Ff', 'N', 'VV', 'Td']].dropna()
df.head(3)

,Местное время в Москве (ВДНХ),T,Po,U,DD,Ff,N,VV,Td
1,31.08.2015 18:00,17.2,747.4,60.0,"Ветер, дующий с северо-запада",1,40%.,10.0,9.3
2,31.08.2015 15:00,18.2,747.2,53.0,"Ветер, дующий с северо-запада",2,70 – 80%.,10.0,8.5
3,31.08.2015 12:00,16.7,747.4,66.0,"Ветер, дующий с северо-северо-запада",2,"90 или более, но не 100%",10.0,10.2


In [305]:
df

,Местное время в Москве (ВДНХ),T,Po,U,DD,Ff,N,VV,Td
1,31.08.2015 18:00,17.2,747.4,60.0,"Ветер, дующий с северо-запада",1,40%.,10.0,9.3
2,31.08.2015 15:00,18.2,747.2,53.0,"Ветер, дующий с северо-запада",2,70 – 80%.,10.0,8.5
3,31.08.2015 12:00,16.7,747.4,66.0,"Ветер, дующий с северо-северо-запада",2,"90 или более, но не 100%",10.0,10.2
4,31.08.2015 09:00,13.7,747.3,80.0,"Ветер, дующий с северо-северо-запада",2,"90 или более, но не 100%",10.0,10.3
5,31.08.2015 06:00,12.2,747.0,89.0,"Ветер, дующий с северо-запада",1,"90 или более, но не 100%",4.0,10.5
...,...,...,...,...,...,...,...,...,...
6502,29.10.2014 12:00,8.2,754.6,40.0,"Штиль, безветрие",0,20–30%.,10.0,-4.8
6503,29.10.2014 11:00,5.6,754.6,49.0,"Ветер, дующий с западо-юго-запада",1,60%.,10.0,-4.3
6504,29.10.2014 10:00,8.3,754.6,44.0,"Ветер, дующий с западо-юго-запада",1,70 – 80%.,10.0,-3.4
6505,29.10.2014 09:00,6.0,754.1,55.0,"Ветер, дующий с запада",1,70 – 80%.,10.0,-2.5


In [306]:
df.dtypes

Местное время в Москве (ВДНХ)     object
T                                float64
Po                               float64
U                                float64
DD                                object
Ff                                 int64
N                                 object
VV                               float64
Td                               float64
dtype: object

In [307]:
np.unique(df.N.dropna())

array(['10%  или менее, но не 0', '100%.', '20–30%.', '40%.', '50%.',
       '60%.', '70 – 80%.', '90  или более, но не 100%', 'Облаков нет.'],
      dtype=object)

Посмотрим, какие занчения принимают категориальные признаки

In [308]:
np.unique(df.DD)

array(['Ветер, дующий с востока',
       'Ветер, дующий с востоко-северо-востока',
       'Ветер, дующий с востоко-юго-востока', 'Ветер, дующий с запада',
       'Ветер, дующий с западо-северо-запада',
       'Ветер, дующий с западо-юго-запада', 'Ветер, дующий с севера',
       'Ветер, дующий с северо-востока', 'Ветер, дующий с северо-запада',
       'Ветер, дующий с северо-северо-востока',
       'Ветер, дующий с северо-северо-запада', 'Ветер, дующий с юга',
       'Ветер, дующий с юго-востока', 'Ветер, дующий с юго-запада',
       'Ветер, дующий с юго-юго-востока',
       'Ветер, дующий с юго-юго-запада', 'Штиль, безветрие'], dtype=object)

In [309]:
np.unique(df.VV)

array([ 0.2,  0.5,  1. ,  2. ,  4. , 10. , 20. ])

Признак N (общая облачность) упорядочить:

In [310]:
mapper_x = {'Облаков нет.': 0 , 
          '10%  или менее, но не 0': 1, 
          '20–30%.': 2, 
          '40%.': 3,
          '50%.': 4,
          '60%.': 5,
          '70 – 80%.': 6,
          '90  или более, но не 100%': 7,
          '100%.': 8}

In [311]:
df.N = df.N.apply(mapper)

Признак VV (горизонтальная дальность видимости) мы решили привести к вещественному виду. Наблюдениям со значением 'менее 0.05' мы даем значение 0.05.

In [312]:
def realizer(x):
    x = str(x)
    if x.split()[0].isalpha():
        return float(x.split()[1])
    else:
        return float(x)

In [313]:
df.VV = df.VV.apply(realizer)

In [314]:
df.head()

,Местное время в Москве (ВДНХ),T,Po,U,DD,Ff,N,VV,Td
1,31.08.2015 18:00,17.2,747.4,60.0,"Ветер, дующий с северо-запада",1,3,10.0,9.3
2,31.08.2015 15:00,18.2,747.2,53.0,"Ветер, дующий с северо-запада",2,6,10.0,8.5
3,31.08.2015 12:00,16.7,747.4,66.0,"Ветер, дующий с северо-северо-запада",2,7,10.0,10.2
4,31.08.2015 09:00,13.7,747.3,80.0,"Ветер, дующий с северо-северо-запада",2,7,10.0,10.3
5,31.08.2015 06:00,12.2,747.0,89.0,"Ветер, дующий с северо-запада",1,7,4.0,10.5


В нашем датасате с погодой для каждого дня присутствуют наблюдения для нескольких временных точек. Мы ограничимся использованием четырех: 9:00, 12:00, 15:00, 18:00. Создадим для них отдельные таблицы

#### 9 часов:

In [315]:
df['Местное время в Москве (ВДНХ)'] = pd.to_datetime(df['Местное время в Москве (ВДНХ)'], format='%d.%m.%Y %H:%M')
df = df.rename(columns={'Местное время в Москве (ВДНХ)': 'date'})

# Фильтрация данных, чтобы оставить только записи со временем 09:00
df_9 = df[df['date'].dt.hour == 9]
df_9 = pd.DataFrame(df_9)
df_9['date'] = df_9['date'].dt.date

df_9.sample(3)

,date,T,Po,U,DD,Ff,N,VV,Td
1173,2015-06-09,18.7,750.7,44.0,"Ветер, дующий с западо-северо-запада",1,2,10.0,6.1
195,2015-08-07,22.4,754.3,51.0,"Ветер, дующий с западо-северо-запада",1,2,10.0,11.6
525,2015-07-06,17.0,747.4,49.0,"Ветер, дующий с северо-северо-запада",3,1,10.0,6.1


#### 12 часов:

In [316]:
df_12 = df[df['date'].dt.hour == 12]
df_12 = pd.DataFrame(df_12)
df_12['date'] = df_12['date'].dt.date

#### 15 часов:

In [317]:
df_15 = df[df['date'].dt.hour == 15]
df_15 = pd.DataFrame(df_15)
df_15['date'] = df_15['date'].dt.date

#### 18 часов:

In [318]:
df_18 = df[df['date'].dt.hour == 18]
df_18 = pd.DataFrame(df_18)
df_18['date'] = df_18['date'].dt.date

### Соединяем данные

Мы объединяем нашу табличку с постами с табличкой с погодой

In [319]:
data = pd.read_csv('lemmatized_no_noise.csv').drop('Unnamed: 0', axis = 1)
data.sample(3)

,author,date,n_likes,n_reposts,with_photo,is_repost?,text
7220,dying,2015-02-09,786,5,False,False,мочь
8564,dying,2015-01-05,538,7,False,False,друг это человек который приносить домой говнище
4816,dying,2015-04-17,2491,8,False,False,окей гугл ... 404 found otebites uje ot menya


In [320]:
data.isna().sum()

author         0
date           0
n_likes        0
n_reposts      0
with_photo     0
is_repost?     0
text          24
dtype: int64

In [321]:
data.sample(3)

,author,date,n_likes,n_reposts,with_photo,is_repost?,text
1333,dying,2015-07-17,750,10,False,False,весь человек человек солнышко
5861,dying,2015-03-20,1626,19,False,False,любить ой это зря
7579,dying,2015-01-31,639,3,False,False,красный глаз затуманивать рассудок судить небо...


In [322]:
data.date = data.date.apply(str) # для простоты мерджа, переведем все даты в формат строки

df_9.date = df_9.date.apply(str)
df_12.date = df_12.date.apply(str)
df_15.date = df_15.date.apply(str)
df_18.date = df_18.date.apply(str)

In [324]:
merged_9 = pd.merge(left = data, right = df_9, on = 'date', how = 'left')
merged_12 = pd.merge(left = data, right = df_12, on = 'date', how = 'left')
merged_15 = pd.merge(left = data, right = df_15, on = 'date', how = 'left')
merged_18 = pd.merge(left = data, right = df_18, on = 'date', how = 'left')

In [326]:
merged_9.to_csv('data_weather_9.csv')
merged_12.to_csv('data_weather_12.csv')
merged_15.to_csv('data_weather_15.csv')
merged_18.to_csv('data_weather_18.csv')